In [ ]:
import numpy as np
from scipy.sparse import csr_matrix
import os
from heapq import nlargest
from random import sample


def load_cooccurrences(path):
  """ Usage: load_cooccurrences("cooccurrence.bin") """
  dt = np.dtype([('i', '<i4'), ('j', '<i4'), ('x', '<f8')])
  arr = np.fromfile(path, dtype=dt)
  return csr_matrix((arr['x'], (arr['i']-1, arr['j']-1)))


def load_vocab(path):
  """
  Usage: load_vocab("vocab.txt")

  Returns a list of tuples of (word: str, freq: int)
  """
  with open(path, "r") as f:
    res = []
    for line in f:
      word, freq = line.split(' ')
      res.append((word, int(freq)))
  return res


def load_vectors(path, vector_size, vocab_size):
  """
  Usage: load_vectors("vectors.bin")

  Returns (word_vectors, context_vectors, word_biases, context_biases).

  word_vectors and context_vectors are (vocab_size, vector_size) matrices
  word_biases and context_biases are (vocab_size) arrays
  """
  dt = np.dtype('<f8')
  arr = np.fromfile(path, dtype=dt)
  vecs = arr.reshape((2*vocab_size, vector_size+1))
  word_mat, ctx_mat = np.split(vecs, 2)
  word, ctx = word_mat[:, :vector_size], ctx_mat[:, :vector_size]
  bias_word, bias_ctx = word_mat[:, vector_size], ctx_mat[:, vector_size]
  return word, ctx, bias_word, bias_ctx


home = os.path.expanduser('~')

# Load sample data: these embeddings were trained on 10% of wikipedia using
# GloVe-tutorial, replicating the "Sub-Wikipedia" dataset.
cooccur_path = "sample-data/sub-wikipedia/cooccurrence.bin"
vocab_path = "sample-data/sub-wikipedia/vocab.txt"
vector_path = "sample-data/sub-wikipedia/vectors.bin"
vector_size = 50

print("Loading...")
C = load_cooccurrences(cooccur_path)
vocab = load_vocab(vocab_path)
dictionary = [v[0] for v in vocab]
freq = [v[1] for v in vocab]
D = len(dictionary)
vecs = load_vectors(vector_path, vector_size, D)
word, ctx, B, B_ctx = vecs

print("Loaded. Generating cooccurrence sums...")
Csum = C.sum(axis=1).A1

print("Done.")


In [ ]:
print("Generating 200 samples of the top 5k pairs...")

top_100k_sample = sample(
    nlargest(5000, range(D), key=lambda i: freq[i]), 200)
top_pairs = list(zip(top_100k_sample[::2], top_100k_sample[1::2]))

print("Done.")


In [ ]:
# TODO: Implement bias corrections for made-up words; see pp. 17-18

# Set to None to disable
# fake = "foobar123"
fake = None

if fake is not None:
  if fake in dictionary:
    raise RuntimeError("Fake word isn't fake!")

  fake_idx = D
  dictionary.append(fake)
  freq.append(0)
  B = np.append(B, 0)
  D += 1
  C.resize((D, D))
else:
  fake_idx = -1


In [ ]:
import numpy as np
import cupy as cp
from numba import cuda, njit
from math import exp, log, sqrt, inf
from bisect import bisect_left
from typing import List
from functools import reduce
import types

Gamma = 5
lambda_ = 5


def gamma(d):
  return 1/d if d <= Gamma else 0

# second_order_cooccur = omega_i for second-order sequences
# NB: In the original paper, second_order_cooccur isn't multiplied by a factor of 2.
# This almost 100% certainly seems to be a mistake, despite being repeated twice.
second_order_cooccur = 2*sum(gamma(d) for d in range(1, lambda_+1))


# make sure we don't accidentally capture any global variables!
# this is meant as a debugging aid and can safely be removed.
# From: https://stackoverflow.com/a/54249582/3398839
def noglobal(f):
  return types.FunctionType(f.__code__, globals().copy(), f.__name__, f.__defaults__, f.__closure__)


@cuda.jit(device=True)
@noglobal
def cuda_model_f(u, v, c, epsilon, B):
  return max(log(c)-B[u]-B[v], epsilon)


@cuda.jit
@noglobal
def sim2_kernel(s, Cs_prime, Ms_norm, Ms_dot_Mt, Mt_rows, Mt_norm, B, T_wgt, res):
  """
  Calculates sim2. This function is JIT-compiled to a CUDA kernel using numba,
  and is meant to be called with D blocks and 32 threads per block. Each THREAD
  BLOCK calculates the value of sim2, weighted by T_wgt, for a single word and
  for all values of delta. The first 30 threads correspond to the first 30
  values of l (ie. delta=(l+1)/5), and the last two are unused. The block index
  is the index of the word to calculate.

  Assumes i != s and i is not a target. Otherwise, the result is incorrect but
  not undefined, so such results must be computed manually with comp_diff_naive
  and not included in the argmax. See comp_diff_naive for a much cleaner, more
  general implementation.

  All column indices below refer to the keep-indices, NOT the indices into the
  full dictionary.

  s: source word
  Cs_prime: row s of C', which includes Dhat
  Ms_dot_Mt[t]: dot product of rows s and t in M (where t is INDEX into T)
  Ms_norm: L2 norm of row s in M
  Mt_rows[t, i]: value of column i of t's row in M (where t is INDEX into T)
  Mt_norm[t]: norm of row t in M (where t is INDEX into T)
  B[i]: bias
  T_wgt: weight for each target index (+1 for POS, -1 for NEG)
  res[i, l]: output matrix
  """
  i = cuda.blockIdx.x
  l = cuda.threadIdx.x
  if i < Cs_prime.shape[0] and l < 30:
    delta = (l+1)/5
    sim2 = 0.
    old_M_si = cuda_model_f(s, i, Cs_prime[i], 0, B)
    new_M_si = cuda_model_f(s, i, Cs_prime[i]+delta, 0, B)
    for t in range(Mt_rows.shape[0]):
      new_Ms_dot_Mt = Ms_dot_Mt[t] + (new_M_si-old_M_si)*Mt_rows[t, i]
      new_Ms_norm = Ms_norm + new_M_si*new_M_si - old_M_si*old_M_si
      old_sim2 = Ms_dot_Mt[t]/sqrt(Ms_norm*Mt_norm[t])
      new_sim2 = new_Ms_dot_Mt/sqrt(new_Ms_norm*Mt_norm[t])
      sim2 += T_wgt[t]*(new_sim2-old_sim2)
    res[i, l] = sim2


@njit
@noglobal
def _model_f(u: int, v: int, c: float, epsilon: float, B: np.ndarray) -> float:
  """ Optimized f() """
  logc = log(c) if c > 0 else -inf
  return max(logc-B[u]-B[v], epsilon)


@njit
@noglobal
def _M(u: int, v: int, C_uv: float, Dhat: np.ndarray, s, B):
  """ Optimized M. Automatically applies Dhat when necessary. """
  if u == s:
    C_offset = Dhat[v]
  elif v == s:
    C_offset = Dhat[u]
  else:
    C_offset = 0
  return _model_f(u, v, C_uv+C_offset, 0, B)


class CorpusPoison:
  class CompDiffState:
    """ Structure to store a few state variables. """
    @noglobal
    def __init__(self, Jhat: float, Csum, M_norms, Ms_dot, Delta_size: int):
      self.Jhat = Jhat
      self.Csum = Csum
      self.M_norms = M_norms
      self.Ms_dot = Ms_dot
      self.Delta_size = Delta_size

  class TensorPrime:
    """
    TensorPrime allows you to make temporary modifications to a vector/matrix.
    The original item is never updated; all modifications are stored in the
    `overrides` dictionary. Call apply() to permamently apply the modifications
    to an item.
    """
    @noglobal
    def __init__(self, tensor, overrides=None):
      self.tensor = tensor
      self.overrides = overrides if overrides is not None else {}

    @noglobal
    def __getitem__(self, key):
      """ Returns the overridden value, defaulting to the original item """
      return self.overrides[key] if key in self.overrides else self.tensor[key]

    @noglobal
    def __setitem__(self, key, value):
      """ Adds the new value to the override dictionary without modfying the underlying item """
      self.overrides[key] = value

    @noglobal
    def apply(self, other):
      """ Applies all overrides to the specified vector """
      for key in self.overrides:
        other[key] = self.overrides[key]

    @noglobal
    def __repr__(self):
      return "OVERRIDES: {" + ", ".join(f"{k}: {self.tensor[k]}->{v}" for k, v in self.overrides.items()) + "}"

  @noglobal
  def __init__(self, dictionary, cooccur, bias, orig_Csum):
    """
    dictionary: list of words as strings
    cooccur: cooccurrence matrix (of type csc_matrix)
    bias: bias vector
    orig_Csum: vector, each element is the sum of a row in the original C matrix
    """
    self.dictionary = dictionary
    self.C = cooccur
    self.B = bias
    self.e60 = exp(-60)
    self.orig_Csum = orig_Csum

  @noglobal
  def model_f(self, u: int, v: int, c: float, epsilon: float, B: np.ndarray) -> float:
    # Thin wrapper for easily adding debug statements
    return _model_f(u, v, c, epsilon, B)

  @noglobal
  def M(self, u, v, C, Dhat=None):
    # Thin wrapper for easily adding debug statements
    Dhat = Dhat if Dhat is not None else self.Dhat
    return _M(u, v, C[u, v], Dhat, self.s, self.B)

  @noglobal
  def sim1(self, t: float, C: float, Csum: List[float], Dhat=None):
    s = self.s
    num = self.M(s, t, C, Dhat)
    den1 = self.model_f(s, t, Csum[s], self.e60, self.B)
    den2 = self.model_f(s, t, Csum[t], self.e60, self.B)
    return num/sqrt(den1*den2)

  @noglobal
  def sim2(self, t, Ms_dot, M_norms):
    s = self.s
    return Ms_dot[t]/sqrt(M_norms[s]*M_norms[t])

  @noglobal
  def comp_diff_naive(self, i: int, delta: float, state: CompDiffState) -> CompDiffState:
    """ Calculate the CompDiffState for the given i, delta. All vectors/matrices are `TensorPrime`s. """
    # note: we assume C is symmetric and s is not in POS or NEG.
    s = self.s

    # calculate the new state after hypothetically executing C[s, i] += delta.
    # TensorPrime is essentially a tensor view that records our modifications
    # without applying them to the original tensor. this makes things much cleaner.
    C = self.TensorPrime(self.C)
    C[s, i] += delta
    if i != s:
      C[i, s] += delta

    # apply all side effects as a result of changes to the cooccurrence counts

    # side effect: Csum[s] += delta and Csum[i] += delta
    Csum = self.TensorPrime(state.Csum)
    Csum[s] += delta
    if i != s:
      Csum[i] += delta

    # side effect: update Ms_dot[t]
    Ms_dot = self.TensorPrime(state.Ms_dot)
    def old_M(u, v): return self.M(u, v, self.C)
    def new_M(u, v): return self.M(u, v, C)
    for t in self.T:
      # account for the change in M_si
      Ms_dot[t] += new_M(s, i)*new_M(t, i) - old_M(s, i)*old_M(t, i)
      if i == t:
        # account for the change in M_is
        Ms_dot[t] += new_M(t, s)*new_M(s, s) - old_M(t, s)*old_M(s, s)

    # side effect: M_norms[s] += change in (M_si)^2
    M_norms = self.TensorPrime(state.M_norms)
    M_norms[s] += new_M(s, i)**2 - old_M(s, i)**2
    if i in self.T:
      M_norms[i] += new_M(i, s)**2 - old_M(i, s)**2

    # ok, the new state is almost ready!
    # calculate the changes in sim1+2 and the objective
    dsim12_total = 0.
    for t in self.T:
      old_sim1 = self.sim1(t, self.C, state.Csum)
      new_sim1 = self.sim1(t, C, Csum)
      old_sim2 = self.sim2(t, state.Ms_dot, state.M_norms)
      new_sim2 = self.sim2(t, Ms_dot, M_norms)
      change_sim12 = ((new_sim1-old_sim1)+(new_sim2-old_sim2))/2
      sign = 1 if t in self.POS else -1
      dsim12_total += sign*change_sim12

    dJhat = dsim12_total/len(self.T)

    # adjust weight for second-order sequences; see p. 19.
    omega_i = 1 if i in self.POS else second_order_cooccur

    return self.CompDiffState(dJhat, Csum, M_norms, Ms_dot, delta/omega_i)

  @noglobal
  def solve_greedy(self, s: int, POS, NEG, t_rank: float, alpha: float, max_Delta: float, calc_Jhat10=False):
    # initialize variables
    self.s = s
    self.POS = POS
    self.NEG = NEG
    self.t_rank = t_rank
    self.alpha = alpha
    self.max_Delta = max_Delta
    self.T = T = POS + NEG
    A = T + [s]
    C = self.C
    B = self.B

    # generate a list of column indices to keep. we ignore any columns
    # whose entries are all zero for each target's row.
    # TODO: this could probably be optimized since all arrays are pre-sorted
    print("Generating keep list...")
    # limit to words with at least one pos. cooccurrence for some r in A
    keep = reduce(np.union1d, (
        C.indices[C.indptr[r]:C.indptr[r+1]]
        for r in A
    ))
    # limit to words with at least one pos. M value for some r in A
    keep = reduce(np.union1d, np.fromiter((
        j
        for j in keep
        if any(self.model_f(r, j, C[r, j], 0, B) > 0 for r in A)
    ), dtype=np.int_))
    # make sure to keep all words in A
    keep = np.union1d(A, keep)
    K = len(keep)

    # list of keep-indices for each word in A
    A_keep_list = [i for i, x in enumerate(keep) if x in A]
    # maps each word in A to its keep-index
    A_keep_idx = {keep[i]: i for i in A_keep_list}

    # debug: print out what % of words are in the keep list
    pct = 100*keep.shape[0]/len(self.dictionary)
    print(f"Keep list length: {keep.shape[0]} ({pct:.2f}%)")

    # calculate initial state
    print("Preparing state...")
    # Dhat = Delta hat, ie. vector to add to row s of C'
    self.Dhat = Dhat = np.zeros(D, dtype=np.float32)
    # don't modify self.orig_Csum!!
    Csum = self.TensorPrime(self.orig_Csum)

    # map each target to its row in M, using keep-indices
    M_row = {t: np.fromiter((self.M(t, k, C)
                            for k in keep), dtype=np.float32) for t in A}

    # map each target to the norm of its row in M
    M_norms = {t: np.linalg.norm(M_row[t]) ** 2 for t in A}
    # map each target to the dot product of rows s and t in M
    Ms_dot = {t: M_row[s].dot(M_row[t]).item() for t in A}
    del M_row  # save some memory!

    # map each target INDEX in T to +1 for POS and -1 for NEG
    T_wgt = np.array([1]*len(POS) + [-1]*len(NEG), dtype=np.float32)

    # calculate the initial objective Jhat
    total_sim12 = 0.
    for t, w in zip(T, T_wgt):
      sim1 = self.sim1(t, C, Csum)
      sim2 = self.sim2(t, Ms_dot, M_norms)
      sim12 = (sim1+sim2)/2
      total_sim12 += w*sim12
    init_Jhat = total_sim12/len(T)
    print("Initial Jhat:", init_Jhat)

    # initialize the state object
    state = self.CompDiffState(init_Jhat, Csum, M_norms, Ms_dot, 0)

    # norm of row s of M
    Ms_norm = M_norms[s]
    # calculate padding Mt_rows to get 32-byte alignment: we have K 4-byte
    # floats, so calculate number of bytes, ceil to multiple of 32, and divide
    # by 4 to get the number of floats to pad with.
    K_aligned = ((4*K+31) & (~31))//4
    pad = K_aligned - K

    # copy data to the CUDA device
    print("Preparing CUDA...")
    # map t's INDEX into T, to the dot product of rows s and t
    Ms_dot_Mt = np.array([Ms_dot[t] for t in T], dtype=np.float32)
    # map t's INDEX into T, to row t of M, limiting to "keep" columns
    Mt_rows = np.array([
        [self.M(t, j, C) for j in keep] + [0]*pad
        for t in T], dtype=np.float32)
    # map each keep-index to it's entry in row s of C'
    sim2_Cs_prime = cuda.to_device(C[s, keep].todense().A1)
    # same as Ms_dot_Mt
    sim2_Ms_dot_Mt = cuda.to_device(Ms_dot_Mt)
    # same as Mt_rows
    sim2_Mt_rows = cuda.to_device(Mt_rows)
    # map t's INDEX into T, to the norm of its row in M
    sim2_Mt_norm = cuda.to_device(
        np.array([M_norms[t] for t in T], dtype=np.float32))
    # map each keep-index to its bias
    sim2_B = cuda.to_device(B[keep])
    # same as T_wgt
    sim2_T_wgt = cuda.to_device(T_wgt)
    # output matrix: sim2_res[i, l] = d_{i,delta}[sim2(s, i)]
    # uses keep-indices for i
    sim2_res = cuda.device_array((K, 32), dtype=np.float32)

    print("Starting iteration...")
    # we only really need 30 threads per block, but all warps are 32 threads.
    # round up to 32 so there is exactly 1 block per warp
    threadsperblock = 32
    blockspergrid = K
    iters = 0
    while state.Jhat < t_rank + alpha and state.Delta_size < max_Delta:
      iters += 1

      # call the kernel. note: the kernel always uses keep-indices or T indices
      sim2_kernel[blockspergrid, threadsperblock](
          A_keep_idx[s], sim2_Cs_prime, Ms_norm, sim2_Ms_dot_Mt, sim2_Mt_rows,
          sim2_Mt_norm, sim2_B, sim2_T_wgt, sim2_res)
      sim2_cupy = cp.asarray(sim2_res)

      # override all words in A so they won't be picked up by argmax. words in
      # A may be computed incorrectly by the kernel, so we must use calculate
      # naively with comp_diff_naive.
      sim2_cupy[A_keep_list, :] = -100

      # argmax the remaining sim2 values, resulting in one keep-index for each
      # l/delta. we still need to account for sim1, which varies depending on
      # delta, so we can't just argmax the whole matrix at once.
      argmax_by_l = sim2_cupy.argmax(axis=0)

      # for each delta, run comp_diff_naive for the index given by argmax, and
      # all indices in A. store the results in dmap
      dmap = {}
      for l in range(30):
        delta = (l+1)/5
        i = keep[argmax_by_l[l].item()]
        for j in A + [i]:
          dmap[(j, delta)] = self.comp_diff_naive(j, delta, state)

      # find the argmax among ALL entries in dmap
      i_star, delta_star = -1, -1
      best = -inf
      for i, delta in dmap:
        cd_state = dmap[(i, delta)]
        score = cd_state.Jhat / cd_state.Delta_size
        if score > best:
          best = score
          i_star, delta_star = i, delta
      # get the CompDiffState for i_star and delta_star
      cd_star = dmap[(i_star, delta_star)]

      # at this point, we accept i_star and delta_star!

      # Update naive state
      state.Jhat += cd_star.Jhat
      cd_star.Csum.apply(state.Csum)
      cd_star.M_norms.apply(state.M_norms)
      cd_star.Ms_dot.apply(state.Ms_dot)
      state.Delta_size += cd_star.Delta_size
      Dhat[i_star] += delta_star

      # Update CUDA state
      # we need to apply some things manually to account for keep-indices
      # find the keep-index of i_star
      i_star_keep = bisect_left(keep, i_star)
      if i_star_keep >= K or keep[i_star_keep] != i_star:
        print("UNABLE TO FIND i_star in keep!")
        return None
      sim2_Cs_prime[i_star_keep] += delta_star
      for t, x in cd_star.Ms_dot.overrides.items():
        if t in T:
          sim2_Ms_dot_Mt[T.index(t)] = x
      Ms_norm = cd_star.M_norms[s]
      # not necessary since we calculate words in A naively:
      # if i_star in T:
      #   sim2_Mt_rows[T.index(i_star), A_keep_idx[s]] += delta_star
      for t, x in cd_star.M_norms.overrides.items():
        if t in T:
          sim2_Mt_norm[T.index(t)] = x

      # print a status update every 100 iterations
      if iters % 100 == 0:
        print("Iterated!", iters, i_star, delta_star, state.Jhat,
              state.Delta_size, dictionary[i_star])

    print("Done!")
    print("Total iterations:", iters)
    print("Final objective:", state.Jhat)
    print("Increase in objective:", state.Jhat - init_Jhat)

    if calc_Jhat10:
      # calculate increase in objective if we set Dhat <- 10*Dhat
      # (see bottom of p. 9)
      Dhat10 = Dhat*10
      M_row10 = {t: np.fromiter((self.M(t, k, C, Dhat10)
                                for k in keep), dtype=np.float32) for t in A}
      M_norms10 = {t: np.linalg.norm(M_row10[t]) ** 2 for t in A}
      Ms_dot10 = {t: M_row10[s].dot(M_row10[t]).item() for t in A}
      del M_row10  # save some memory!
      Csum10 = {t: self.orig_Csum[t]+Dhat10[t] for t in T}
      Csum10[s] = self.orig_Csum[s]+np.sum(Dhat10)
      total_sim12 = 0.
      for t, w in zip(T, T_wgt):
        sim1 = self.sim1(t, C, Csum10, Dhat10)
        sim2 = self.sim2(t, Ms_dot10, M_norms10)
        sim12 = (sim1+sim2)/2
        total_sim12 += w*sim12
      Jhat10 = total_sim12/len(T)
      Jhat_res = Jhat10
      print("10xDhat Final objective:", Jhat10)
      print("10xDhat Increase in objective:", Jhat10 - init_Jhat)
    else:
      Jhat_res = state.Jhat

    print("|Delta|:", state.Delta_size)
    print()
    return Dhat, Jhat_res-init_Jhat


def SIM1(u, v):
  """ SIM1 from GloVe """
  return word[u].dot(ctx[v]) + ctx[u].dot(word[v])


def SIM2(u, v):
  """ SIM2 from GloVe """
  return word[u].dot(word[v]) + ctx[u].dot(ctx[v])


model = CorpusPoison(dictionary, C, B, Csum)
# dJhat10_sampled = []

for s, t in top_pairs[:10]:
  print()
  print("Trying pair:", s, t, dictionary[s], dictionary[t])
  print("Before poisoning:")
  print("SIM1:", SIM1(s, t))
  print("SIM2:", SIM2(s, t))
  print("SIM1+2:", (SIM1(s, t)+SIM2(s, t))/2)

  # Following p. 9, use max_Delta <- 1250/10 then set Dhat <- Dhat*10
  # Dhat, dJhat10 = model.solve_greedy(
  #     s, POS=[t], NEG=[], t_rank=inf, alpha=0, max_Delta=125, calc_Jhat10=True)
  Dhat, _ = model.solve_greedy(
      s, POS=[t], NEG=[], t_rank=inf, alpha=0, max_Delta=125, calc_Jhat10=False)
  # dJhat10_sampled.append(dJhat10)
  Dhat *= 10

  print()
  print("Top 30 words:")
  for i in nlargest(30, range(D), key=lambda i: Dhat[i]):
    print(f"{i}, {dictionary[i]}: {Dhat[i]}")

  np.save(f"Dhat_{dictionary[s]}_{dictionary[t]}.npy", Dhat)

# print("dJhat10s", dJhat10_sampled)
# print("dJhat10s mean", np.mean(dJhat10_sampled))


In [ ]:
import random
from math import ceil


def place_additions(Dhat, s, POS):
  """ Algorithm 2, Placement into corpus: finding the change set ∆ """
  Delta = []

  for t in POS:
    Delta += [(s, t)] * ceil(Dhat[t]/gamma(1))

  change_map = {u: Du for u, Du in enumerate(Dhat) if Du != 0 and u not in POS}

  # note: second_order_cooccur is defined much earlier in the file.
  min_sequences_required = ceil(sum(change_map.values())/second_order_cooccur)
  default_seq = [-1]*lambda_ + [s] + [-1]*lambda_
  live = [default_seq[:] for _ in range(min_sequences_required)]
  indices = list(range(lambda_)) + list(range(lambda_+1, 2*lambda_+1))

  cm_keys = list(change_map.keys())
  for u in cm_keys:
    while change_map[u] > 0:
      best_seq_i = best_i = -1
      best = inf
      for seq_i, seq in enumerate(live):
        cm_u = change_map[u]
        for i in indices:
          if seq[i] < 0:
            score = abs(gamma(abs(lambda_-i))-cm_u)
            if score < best:
              best_seq_i, best_i = seq_i, i
              best = score
      seq, i = live[best_seq_i], best_i

      seq[i] = u
      change_map[u] -= gamma(abs(lambda_-i))
      if all(seq[i] >= 0 for i in indices):
        Delta.append(seq)
        # remove seq from live
        if best_seq_i < len(live)-1:
          live[best_seq_i] = live.pop()
        else:
          live.pop()
      if not live:
        live = [default_seq[:]]

  for seq in live:
    for i in indices:
      if seq[i] < 0:
        seq[i] = random.choice(cm_keys)
    Delta.append(seq)

  return Delta


s, t = top_pairs[0]
print("Source:", dictionary[s])
print("Target:", dictionary[t])
print("Running placement algorithm...")
Delta = place_additions(Dhat, s, [t])

# print a few example sequences
print("Done! Sample sequences:")
for seq in Delta[::len(Delta)//20]:
  print([dictionary[w] for w in seq])

with open(f"Delta_{dictionary[s]}_{dictionary[t]}.txt", "w") as f:
  for seq in Delta:
    f.write(" ".join(dictionary[w] for w in seq) + "\n")
